<a href="https://colab.research.google.com/github/tusker4/Sesac_Saltlux_DeepLearning/blob/main/7_NLP_%EB%A7%90%EB%AD%89%EC%B9%98%EB%A1%9C%EB%B6%80%ED%84%B0%ED%86%A0%ED%94%BD%EC%B6%94%EC%B6%9C)_Bertopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

https://maartengr.github.io/BERTopic

- Bert, TF-IDF(카운트 기반으로 중요도 표시 -> topic 추출의 재료) 를 활용하여서, 주제 설명에 중요한 단어를 추출 클러스터링 기반 토픽 모델링 기술
- 특징
  - 써드파트 모듈
  - 텍스트 데이터 -> SBert 로 임베딩 수행 (자연어 -> 벡터화)
  - 문서를 군집화 처리
    - UMAP 으로 임베딩 차원을 줄이고
    - HDBSCAN으로 차원축소 임베딩을 한 재료를 클러스터링 처리 수행 -> 유사한 문서들을 군집처리함
  - 토픽표현 생성
    - TD-IDF 기반으로 토픽 추출
     

In [ ]:
!pip install -q bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
# 시각화도구
!pip install -q bertopic[visualization]


In [ ]:
# 형태소 분석기
!pip install koNlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 38.7 MB/s eta 0:00:00


# 데이터
- https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt

In [ ]:
!wget https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt

--2023-11-14 04:05:02--  https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43694449 (42M) [text/plain]
Saving to: ‘2016-10-20.txt’

2016-10-20.txt      100%[===================>]  41.67M   184MB/s    in 0.2s    

2023-11-14 04:05:02 (184 MB/s) - ‘2016-10-20.txt’ saved [43694449/43694449]



In [ ]:
!head -n 5  2016-10-20.txt

 
19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성

In [ ]:
text_data_file = '2016-10-20.txt'

# 데이터 준비(전처리)

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Hannanum
from bertopic import BERTopic

In [ ]:
# 파일에서 라인별로 읽어서 하나씩 추출하여 리스트에 담는다
docs = [ line.strip()
        for line in open(text_data_file, encoding='utf-8').readlines()
         # 내용이 있으면서 숫자만 있는 문자열이 아닌것만 추출
         if line.strip() and not line.strip().replace(' ', '').isdecimal()
        ]
len(docs)

27540

In [ ]:
#  전처리 , 빈 문자열 -> 제거 , 숫자만 있는 줄

In [ ]:
docs

# BerTopic 모델 생성
- sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens

In [ ]:
class CusTokenizer:
  # 분절행위 수행 -> 기반 형태소 분석 -> 토큰 생성, 토큰의 길이가 존재하는 값만 적용
  def __init__(self, tagging):
    self.tagging = tagging   # Hannanum, Okt, ....

  # 클래스를 함수처럼 사용할 수 있게 처리
  def __call__(self, sentence):
    sentence = sentence[:10000] # 설정(개수를 줄이겠다, 학습시간이 걸림)
    tokens = self.tagging.morphs( sentence )  # 토큰 획득
    return [ token for token in tokens if len(token) > 1]  #  한글자 토큰 포함 X

    pass

cus_tokenizer = CusTokenizer(Hannanum())
vectorizer = CountVectorizer(tokenizer = cus_tokenizer  , max_features = 300)

In [ ]:
repo = 'sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens'
model = BERTopic(embedding_model = repo,  # BERT
                 vectorizer_model = vectorizer,      # 백터 => 카운트 내용 적용
                 nr_topics       = 20  , # 얻고자하는 토픽의 개수
                 top_n_words      =  10,    # 추출한 토픽당 필요한 단어수(토픽추출을 위한 재료를 몇개까지 사용?)
                 calculate_probabilities = True  # 토픽 확률도 같이 계산해서 노출

                 )

In [ ]:
# 토픽과 각각의 확률값 = > 20개 나온다
topics, probs = model.fit_transform(docs)

KeyboardInterrupt: ignored

In [ ]:
# 저장위치
# /content/drive/MyDrive/머신러닝(솔트룩스)/딥 러닝/딥러닝_NLP
save_path = /content/drive/MyDrive/머신러닝(솔트룩스)/딥 러닝/딥러닝_NLP/bertopic_model
model.save(save_path)
# 모델 로드
BERTopic.load(save_path)

In [ ]:
# 특정 문장의 토픽구하기
topics_, probs_ = model.transform( [ '신규 긴글의 문장'])

# 시각화

In [ ]:
# 토픽 정보 출력
model.get_topic_info()
